In [1]:
import spacy
import nltk
from nltk.stem.porter import *
import openpyxl
from google.colab import drive
import re
from typing import List
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

In [11]:
import pandas as pd

In [2]:
pip install numerizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for numerizer: filename=numerizer-0.2.2-py2.py3-none-any.whl size=7096 sha256=1851edae20ea00ed2a81b59368ece3b4b57c9a131b47c2932a97fdc5d27c6950
  Stored in directory: /root/.cache/pip/wheels/82/25/e1/0703d1e080583335d8f17af101f82dba34ba8c4148d71ad782
Successfully built numerizer


In [3]:
from numerizer import numerize


# import data

In [81]:
drive.mount('/content/drive')
book = openpyxl.load_workbook('/content/drive/MyDrive/dataset.xlsx')
sheet = book.active

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# PreP

In [82]:
import re
from typing import List
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
from numerizer import numerize

class SpacyPreprocessor:
    def __init__(
        self,
        spacy_model=None,
        remove_special=True,
        pos_to_remove=None,
        remove_stopwords=False,
        lemmatize=False,
    ):
       
        
        self._pos_to_remove = pos_to_remove
        self._remove_stopwords = remove_stopwords
        self._remove_special = remove_special
        self._lemmatize = lemmatize

        if not spacy_model:
            self.model = spacy.load("en_core_web_sm")
        else:
            self.model = spacy_model

    @staticmethod
    def download_spacy_model(model="en_core_web_sm"):
        print(f"Downloading spaCy model {model}")
        spacy.cli.download(model)
        print(f"Finished downloading model")

    @staticmethod
    def load_model(model="en_core_web_sm"):
        return spacy.load(model, disable=["ner", "parser"])

    def tokenize(self, text) -> List[str]:
 
        doc = self.model(text)
        return [token.text for token in doc]

    def preprocess_text(self, text) -> str:      
        doc = self.model(text)
        return self.__clean(doc)

    def preprocess_text_list(self, texts=List[str]) -> List[str]:
    
        clean_texts = []
        for doc in tqdm(self.model.pipe(texts)):
            clean_texts.append(self.__clean(doc))

        return clean_texts

    def __clean(self, doc: Doc) -> str:

        tokens = []
        # POS Tags removal
        if self._pos_to_remove:
            for token in doc:
                if token.pos_ not in self._pos_to_remove:
                    tokens.append(token)
        else:
            tokens = doc

        # Remove Stopwords
        if self._remove_stopwords:
            tokens = [token for token in tokens if not token.is_stop]

        # remove unwanted tokens
        tokens = [
            token
            for token in tokens
            if not (
                token.is_punct or token.is_space or token.is_quote or token.is_bracket
            )
        ]

        # Remove empty tokens
        tokens = [token for token in tokens if token.text.strip() != ""]

        # Lemmatize
        if self._lemmatize:
            text = " ".join([token.lemma_ for token in tokens])
        else:
            text = " ".join([token.text for token in tokens])

        if self._remove_special:
            # Remove non characters
            text = re.sub(r"[^a-zA-Z0-9\']", " ", text)
        # remove non-Unicode characters
        text = re.sub(r"[^\x00-\x7F]+", "", text)
        text = text.lower()
      

        return text



if __name__ == "__main__":
    spacy_model = SpacyPreprocessor.load_model()
    preprocessor = SpacyPreprocessor(spacy_model=spacy_model, lemmatize=True)
    stemmer = PorterStemmer()

for row in sheet.iter_cols(min_row=2 , min_col=1, max_row=148 , max_col=1):
    for cell in row:
      t =[]
      result = ''
      angulation = []
      
      for sent in nlp(cell.value).sents:
        clean_text = preprocessor.preprocess_text(str(sent))


        
      
        if re.search('mesio.*(angular)*', clean_text):
            result = result + 'mesioangular'
            c2 = sheet.cell(row=cell.row , column = 7)
            c2.value = 'mesioangular'
        
        
        elif re.search(stemmer.stem('horizontal'), clean_text):
                result = result + 'horizontal'
                c2 = sheet.cell(row=cell.row , column = 7)
                c2.value = 'horizontal'
  
        
        elif  re.search(stemmer.stem('vertical'), str(clean_text)):
             result = result + 'vertical'
             c2 = sheet.cell(row=cell.row , column = 7)
             c2.value = 'vertical' 

        
        elif re.search(stemmer.stem('distoangular'), str(clean_text)):
            result = result + 'distoangular'
            c2 = sheet.cell(row=cell.row , column = 7)
            c2.value = 'distoangular'

        #else:
        #   c2 = sheet.cell(row=cell.row , column = 7)
        #   c2.value = ''

        

        #****************************     
        

        curv_incomplete = ['incomplete', 'open apex' ,'develop' , 'not completely formed' ]
        curvature =[]                                   
        for token in curv_incomplete:
          curvature.append(stemmer.stem(token))
        for token in curvature:
          if re.search( token ,  clean_text):
            ttt=token
            result = result + ' ' + 'incomplete'
            c2 = sheet.cell(row=cell.row , column = 9)
            c2.value = 'incomplete'


        dil = ['dilaceration' , 'curvature' ,'caurve' ]
        dilacer =[]
        for token in dil:
          dilacer.append(stemmer.stem(token))
        
        if re.search('(no)+.*(dilaceration)', clean_text):
          result = result + ' ' + 'straight' 
          c2 = sheet.cell(row=cell.row , column =9)
          c2.value = 'straight'

        else:
          for token in dilacer:
            dlc= re.search( token , clean_text)
            if re.search( token , clean_text) :
              result = result + ' ' + 'dilaceration'
              c2 = sheet.cell(row=cell.row , column =9)
              c2.value = 'dilaceration'
            
        c2 = sheet.cell(row=cell.row , column =9)
        if c2.value !='straight' and c2.value !='dilaceration' and  c2.value !='incomplete':
         c2.value = 'straight'
          

        
        cc=[ 'associate','association','contact','not be follow','no cortex' ]

        if re.search( stemmer.stem('no association') , clean_text):
          result = result + ' ' + 'far'
          c2 = sheet.cell(row=cell.row , column =10)
          c2.value = 'far'
         
        elif re.search( stemmer.stem('not associated') , clean_text):
          result = result + ' ' + 'far'
          c2 = sheet.cell(row=cell.row , column =10)
          c2.value = 'far'
          
        elif re.search( 'far from | any contact | not in contact | safe distance' , clean_text):
          result = result + ' ' + 'far'
          c2 = sheet.cell(row=cell.row , column =10)
          c2.value = 'far'
    
        elif re.search(  stemmer.stem('close proximity | proximity ') , clean_text):
          result = result + ' ' + 'proximity'
          c2 = sheet.cell(row=cell.row , column =10)
          c2.value = 'proximity'

        elif(not re.search( stemmer.stem('no association'), clean_text) and not
          re.search( stemmer.stem('not associated') , clean_text) and not 
          re.search( 'far from | any contact | not in contact | safe distance' , clean_text) and not 
          re.search(  stemmer.stem('close proximity') , clean_text) ):

          for token in cc:
            if re.search( token , clean_text):
              c2 = sheet.cell(row=cell.row , column =10)
              c2.value = 'contact'
              if 'contact' not in result:
                result = result + ' ' + 'contact'


        #else:
        #  c2 = sheet.cell(row=cell.row , column =10)
        # c2.value = ''

    
#*********

        if "cm" or "mm" not in clean_text:
          n = re.search('([1-4]|one|two|three)(?=.*(root(s)*))', clean_text)
          if n:
            tt = numerize(n.group(0))
            result = result + ' ' +tt
            c2 = sheet.cell(row=cell.row , column =8)
            c2.value = tt



In [83]:
#setTitle
c = sheet.cell(row=1 , column =7)
c.value = 'm'
c = sheet.cell(row=1 , column =8)
c.value = 'n'
c = sheet.cell(row=1 , column =9)
c.value = 'c'
c = sheet.cell(row=1 , column =10)
c.value = 'r'
c = sheet.cell(row=1 , column =11)
c.value = 'score'

In [84]:
for row in sheet.iter_cols(min_row=2, min_col=2, max_row=148, max_col=2):
    for cell in row:    
        cel = sheet.cell(row=cell.row , column =11)
        if cell.value <= 7 :
            cel.value = 'D1'
        elif cell.value >= 8 and  cell.value <= 9 :
            cel.value = 'D2'
        elif cell.value >= 10 and  cell.value <= 11 :
            cel.value = 'D3'
        elif cell.value >= 12 and  cell.value <= 14 :
            cel.value = 'D4'

In [85]:
book.save('/content/drive/MyDrive/dataset.xlsx')


In [87]:
df = pd.read_excel('/content/drive/MyDrive/dataset.xlsx')
df

,Report,score,Date,Unnamed: 3,Unnamed: 4,Unnamed: 5,m,n,c,r,score.1
0,The left mandibular third molar tooth has 2 ro...,9,98.08.27,NaN,NaN,NaN,mesioangular,2,dilaceration,contact,D2
1,In the right side:\nThe tooth has two roots.\n...,11,07.10.98,NaN,NaN,NaN,vertical,2,dilaceration,contact,D3
2,In the left side:\nThe tooth has two roots.\nT...,10,07.10.99,NaN,NaN,NaN,horizontal,2,dilaceration,contact,D3
3,The tooth has two roots.\nThere is no signific...,6,98.10.14,NaN,NaN,NaN,mesioangular,2,straight,far,D1
4,In the right sides:\nThere are three root apic...,12,98.11.21,NaN,NaN,NaN,distoangular,3,straight,contact,D4
...,...,...,...,...,...,...,...,...,...,...,...
142,The tooth has two roots.\nThere is no signific...,8,99.3.21,NaN,NaN,NaN,mesioangular,2,straight,contact,D2
143,The tooth has two fused roots.\nThere is dilac...,12,98.11.27,NaN,NaN,NaN,distoangular,2,dilaceration,contact,D4
144,The third molars are oriented in horizontal fa...,8,1400.2.22,NaN,NaN,NaN,horizontal,1,straight,contact,D2
145,In both sides there are two roots.\nNo resorp...,8,99.11.4,NaN,NaN,NaN,mesioangular,2,straight,contact,D2
